In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
import time

print("Strongest ML Models for Fraud Detection - Comprehensive Comparison")
print("=" * 70)

# Load data
data = pd.read_csv('new_data.csv')
print(f"Data size: {data.shape}")
print(f"Fraud percentage: {data['Is_Fraud'].mean()*100:.2f}%")

# Advanced feature engineering
def create_advanced_features(df):
    """Create advanced features for fraud detection"""
    df_feat = df.copy()

    # Convert date and time
    df_feat['Transaction_DateTime'] = pd.to_datetime(df_feat['Transaction_Date'] + ' ' + df_feat['Transaction_Time'])

    # Time features
    df_feat['hour'] = df_feat['Transaction_DateTime'].dt.hour
    df_feat['day_of_week'] = df_feat['Transaction_DateTime'].dt.dayofweek
    df_feat['is_weekend'] = df_feat['day_of_week'].isin([5, 6]).astype(int)
    df_feat['day_of_month'] = df_feat['Transaction_DateTime'].dt.day
    df_feat['month'] = df_feat['Transaction_DateTime'].dt.month

    # Time periods
    df_feat['is_night'] = ((df_feat['hour'] >= 22) | (df_feat['hour'] <= 6)).astype(int)
    df_feat['is_business_hours'] = ((df_feat['hour'] >= 9) & (df_feat['hour'] <= 17)).astype(int)
    df_feat['is_morning'] = ((df_feat['hour'] >= 6) & (df_feat['hour'] <= 12)).astype(int)
    df_feat['is_evening'] = ((df_feat['hour'] >= 18) & (df_feat['hour'] <= 23)).astype(int)

    # Customer statistics (without data leakage)
    customer_stats = df_feat.groupby('Account_ID').agg({
        'Transaction_Amount': ['mean', 'std', 'count', 'max', 'min'],
        'Merchant_ID': 'nunique',
        'Transaction_Type': 'nunique',
        'Location_ID': 'nunique'
    }).fillna(0)
    customer_stats.columns = ['cust_avg_amount', 'cust_std_amount', 'cust_trans_count',
                             'cust_max_amount', 'cust_min_amount', 'cust_unique_merchants',
                             'cust_unique_types', 'cust_unique_locations']

    df_feat = df_feat.merge(customer_stats, left_on='Account_ID', right_index=True, how='left')

    # Amount ratios
    df_feat['amount_to_avg_ratio'] = df_feat['Transaction_Amount'] / df_feat['cust_avg_amount']
    df_feat['amount_to_max_ratio'] = df_feat['Transaction_Amount'] / df_feat['cust_max_amount']

    # Handle infinite values
    df_feat['amount_to_avg_ratio'] = df_feat['amount_to_avg_ratio'].replace([np.inf, -np.inf], 10)
    df_feat['amount_to_max_ratio'] = df_feat['amount_to_max_ratio'].replace([np.inf, -np.inf], 10)

    # Standard deviation
    df_feat['amount_deviation'] = (df_feat['Transaction_Amount'] - df_feat['cust_avg_amount']) / df_feat['cust_std_amount']
    df_feat['amount_deviation'] = df_feat['amount_deviation'].replace([np.inf, -np.inf], 0).fillna(0)

    # Behavioral indicators
    df_feat['is_large_transaction'] = (df_feat['Transaction_Amount'] > df_feat['Transaction_Amount'].quantile(0.95)).astype(int)
    df_feat['is_small_transaction'] = (df_feat['Transaction_Amount'] < df_feat['Transaction_Amount'].quantile(0.05)).astype(int)
    df_feat['is_new_customer'] = (df_feat['cust_trans_count'] == 1).astype(int)
    df_feat['is_high_frequency'] = (df_feat['cust_trans_count'] > df_feat['cust_trans_count'].quantile(0.8)).astype(int)

    return df_feat

print("Creating advanced features...")
data_enhanced = create_advanced_features(data)
print(f"Created {data_enhanced.shape[1]} features")

# Prepare data for modeling
features_to_exclude = [
    'Transaction_ID', 'Transaction_Date', 'Transaction_Time', 'Transaction_DateTime',
    'Is_Fraud', 'Account_ID', 'Merchant_ID', 'Device_ID', 'Location_ID',
    'Transaction_Description'
]

feature_columns = [col for col in data_enhanced.columns if col not in features_to_exclude]
X = data_enhanced[feature_columns]
y = data_enhanced['Is_Fraud']

# Encode categorical variables
categorical_columns = X.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

print(f"Final features: {len(feature_columns)}")
print(f"X size: {X.shape}, y size: {y.shape}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training data: {X_train.shape}, Test data: {X_test.shape}")

# Define strongest models
models = {
    'XGBoost': XGBClassifier(
        n_estimators=1000,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1])
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=1000,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        class_weight='balanced',
        verbosity=-1

    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=500,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=500,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    ),
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(
            class_weight='balanced',
            random_state=42,
            max_iter=1000,
            C=0.1,
            solver='liblinear'
        ))
    ]),
    'Neural Network': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', MLPClassifier(
            hidden_layer_sizes=(100, 50),
            alpha=0.01,
            random_state=42,
            max_iter=1000,
            early_stopping=True
        ))
    ])
}

# Train and evaluate models
print("\nTraining strongest models...")
print("=" * 70)

results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n--- Training {name} ---")
    start_time = time.time()

    try:
        # Training
        model.fit(X_train, y_train)

        # Predictions
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = model.decision_function(X_test)
            y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())

        y_pred = model.predict(X_test)

        # Cross-validation
        if name in ['Logistic Regression', 'Neural Network']:
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
        else:
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')

        training_time = time.time() - start_time

        # Calculate metrics
        auc = roc_auc_score(y_test, y_pred_proba)
        pr_auc = average_precision_score(y_test, y_pred_proba)
        accuracy = (y_pred == y_test).mean()

        # Store results
        results[name] = {
            'model': model,
            'auc': auc,
            'pr_auc': pr_auc,
            'accuracy': accuracy,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_pred_proba': y_pred_proba,
            'y_pred': y_pred,
            'training_time': training_time
        }

        print(f"Success {name}")
        print(f"   AUC: {auc:.4f}")
        print(f"   PR-AUC: {pr_auc:.4f}")
        print(f"   Accuracy: {accuracy:.4f}")
        print(f"   CV AUC: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
        print(f"   Time: {training_time:.2f}s")

    except Exception as e:
        print(f"Failed {name}: {str(e)}")
        results[name] = None

# Compare models
print("\n" + "=" * 70)
print("Comprehensive Model Comparison")
print("=" * 70)

comparison_data = []
for name, result in results.items():
    if result is not None:
        comparison_data.append({
            'Model': name,
            'AUC': result['auc'],
            'PR-AUC': result['pr_auc'],
            'Accuracy': result['accuracy'],
            'CV AUC': result['cv_mean'],
            'CV Std': result['cv_std'],
            'Training Time (s)': result['training_time']
        })

comparison_df = pd.DataFrame(comparison_data).sort_values('AUC', ascending=False)
print(comparison_df.round(4))

# Best model
best_model_name = comparison_df.iloc[0]['Model']
best_result = results[best_model_name]

print(f"\nBest model: {best_model_name}")
print(f"   AUC: {best_result['auc']:.4f}")
print(f"   PR-AUC: {best_result['pr_auc']:.4f}")

# Visualization
print("\nCreating comparison charts...")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comprehensive Comparison of Strongest Fraud Detection Models', fontsize=16, fontweight='bold')

# Chart 1: AUC Comparison
ax1 = axes[0, 0]
models_plot = [m for m in results.keys() if results[m] is not None]
auc_scores = [results[m]['auc'] for m in models_plot]
colors = plt.cm.Set3(np.linspace(0, 1, len(models_plot)))

bars = ax1.bar(models_plot, auc_scores, color=colors, alpha=0.8)
ax1.set_title('AUC Score Comparison', fontweight='bold', fontsize=12)
ax1.set_ylabel('AUC Score')
ax1.tick_params(axis='x', rotation=45)
ax1.set_ylim(0, 1)
for bar, score in zip(bars, auc_scores):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

# Chart 2: PR-AUC Comparison
ax2 = axes[0, 1]
pr_scores = [results[m]['pr_auc'] for m in models_plot]
bars = ax2.bar(models_plot, pr_scores, color=colors, alpha=0.8)
ax2.set_title('PR-AUC Score Comparison', fontweight='bold', fontsize=12)
ax2.set_ylabel('PR-AUC Score')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim(0, 1)
for bar, score in zip(bars, pr_scores):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{score:.3f}', ha='center', va='bottom')

# Chart 3: Training Time Comparison
ax3 = axes[0, 2]
times = [results[m]['training_time'] for m in models_plot]
bars = ax3.bar(models_plot, times, color=colors, alpha=0.8)
ax3.set_title('Training Time Comparison', fontweight='bold', fontsize=12)
ax3.set_ylabel('Time (seconds)')
ax3.tick_params(axis='x', rotation=45)
for bar, time_val in zip(bars, times):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{time_val:.1f}s', ha='center', va='bottom')

# Chart 4: ROC Curve
ax4 = axes[1, 0]
from sklearn.metrics import roc_curve

for name, result in results.items():
    if result is not None:
        fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
        ax4.plot(fpr, tpr, label=f'{name} (AUC={result["auc"]:.3f})', linewidth=2)

ax4.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random')
ax4.set_xlabel('False Positive Rate')
ax4.set_ylabel('True Positive Rate')
ax4.set_title('ROC Curve', fontweight='bold', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)

# Chart 5: Precision-Recall Curve
ax5 = axes[1, 1]
for name, result in results.items():
    if result is not None:
        precision, recall, _ = precision_recall_curve(y_test, result['y_pred_proba'])
        ax5.plot(recall, precision, label=f'{name} (AP={result["pr_auc"]:.3f})', linewidth=2)

ax5.set_xlabel('Recall')
ax5.set_ylabel('Precision')
ax5.set_title('Precision-Recall Curve', fontweight='bold', fontsize=12)
ax5.legend()
ax5.grid(True, alpha=0.3)

# Chart 6: Confusion Matrix for Best Model
ax6 = axes[1, 2]
cm = confusion_matrix(y_test, best_result['y_pred'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax6,
            xticklabels=['Not Fraud', 'Fraud'],
            yticklabels=['Not Fraud', 'Fraud'])
ax6.set_title(f'Confusion Matrix - {best_model_name}\n(AUC: {best_result["auc"]:.3f})',
              fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

# Business Performance Analysis
print("\n" + "=" * 70)
print("Business Performance Analysis for Best Model")
print("=" * 70)

def business_performance_analysis(y_true, y_pred_proba, model_name):
    """Analyze business performance of the model"""

    budgets = [100, 500, 1000, 2000, 5000]
    performance_data = []

    for budget in budgets:
        n_review = min(budget, len(y_true))
        top_indices = np.argsort(y_pred_proba)[-n_review:]

        detected_frauds = y_true.iloc[top_indices].sum()
        total_frauds = y_true.sum()

        detection_rate = detected_frauds / total_frauds if total_frauds > 0 else 0
        precision = detected_frauds / n_review if n_review > 0 else 0
        random_detection = n_review / len(y_true)
        improvement = detection_rate / random_detection if random_detection > 0 else 0

        performance_data.append({
            'Budget': budget,
            'Detection Rate': detection_rate,
            'Precision': precision,
            'Improvement': improvement,
            'Frauds Detected': detected_frauds
        })

    return pd.DataFrame(performance_data)

# Performance analysis for best model
business_results = business_performance_analysis(
    pd.Series(y_test), best_result['y_pred_proba'], best_model_name
)

print(f"Performance of {best_model_name} with different budgets:")
for _, row in business_results.iterrows():
    print(f"  Budget {row['Budget']:,}:")
    print(f"    • Detection Rate: {row['Detection Rate']:.1%}")
    print(f"    • Precision: {row['Precision']:.1%}")
    print(f"    • Improvement: {row['Improvement']:.1f}x")
    print(f"    • Frauds Detected: {int(row['Frauds Detected'])}")

# Final recommendations
print("\n" + "=" * 70)
print("Final Recommendations")
print("=" * 70)

print(f"Recommended Model: {best_model_name}")
print(f"Performance: AUC = {best_result['auc']:.4f}")

if best_result['auc'] > 0.8:
    print("Excellent performance - Ready for practical application")
elif best_result['auc'] > 0.7:
    print("Good performance - Needs minor improvement")
elif best_result['auc'] > 0.6:
    print("Acceptable performance - Needs more improvement")
else:
    print("Weak performance - Needs feature re-engineering")

print(f"\nImprovement tips:")
print(f"   • Try data balancing techniques (SMOTE, ADASYN)")
print(f"   • Add more advanced features")
print(f"   • Fine-tune hyperparameters")
print(f"   • Try ensemble methods")

print("=" * 70)

Strongest ML Models for Fraud Detection - Comprehensive Comparison
Data size: (200000, 11)
Fraud percentage: 5.04%
Creating advanced features...
Created 36 features
Final features: 26
X size: (200000, 26), y size: (200000,)
Training data: (140000, 26), Test data: (60000, 26)

Training strongest models...

--- Training XGBoost ---
Success XGBoost
   AUC: 0.5038
   PR-AUC: 0.0508
   Accuracy: 0.8165
   CV AUC: 0.4947 (±0.0064)
   Time: 144.40s

--- Training LightGBM ---
Success LightGBM
   AUC: 0.5100
   PR-AUC: 0.0524
   Accuracy: 0.6833
   CV AUC: 0.4969 (±0.0110)
   Time: 78.97s

--- Training Random Forest ---
Success Random Forest
   AUC: 0.5018
   PR-AUC: 0.0520
   Accuracy: 0.9162
   CV AUC: 0.4942 (±0.0074)
   Time: 794.38s

--- Training Gradient Boosting ---
Success Gradient Boosting
   AUC: 0.5057
   PR-AUC: 0.0512
   Accuracy: 0.9494
   CV AUC: 0.4983 (±0.0074)
   Time: 3599.89s

--- Training Logistic Regression ---
Success Logistic Regression
   AUC: 0.4978
   PR-AUC: 0.0508
 


Business Performance Analysis for Best Model
Performance of LightGBM with different budgets:
  Budget 100.0:
    • Detection Rate: 0.2%
    • Precision: 6.0%
    • Improvement: 1.2x
    • Frauds Detected: 6
  Budget 500.0:
    • Detection Rate: 1.1%
    • Precision: 6.6%
    • Improvement: 1.3x
    • Frauds Detected: 33
  Budget 1,000.0:
    • Detection Rate: 1.9%
    • Precision: 5.6%
    • Improvement: 1.1x
    • Frauds Detected: 56
  Budget 2,000.0:
    • Detection Rate: 3.6%
    • Precision: 5.4%
    • Improvement: 1.1x
    • Frauds Detected: 108
  Budget 5,000.0:
    • Detection Rate: 8.9%
    • Precision: 5.4%
    • Improvement: 1.1x
    • Frauds Detected: 268

Final Recommendations
Recommended Model: LightGBM
Performance: AUC = 0.5100
Weak performance - Needs feature re-engineering

Improvement tips:
   • Try data balancing techniques (SMOTE, ADASYN)
   • Add more advanced features
   • Fine-tune hyperparameters
   • Try ensemble methods
